# Browser-use: 자율 탐색 도구 마스터하기

이번 실습에서 메인으로 다룰 무기는 **Browser-use**라는 오픈소스 프레임워크입니다. (공식 GitHub: [browser-use](https://github.com/browser-use/browser-use))

Browser-use는 에이전트가 사람처럼 웹 브라우저를 직접 제어할 수 있게 해주는 파이썬 라이브러리입니다. 단순히 데이터를 긁어오는 '크롤링'을 넘어, AI가 웹사이트의 버튼을 클릭하고, 정보를 검색하고, 복잡한 워크플로우를 스스로 수행하게 만드는 웹 에이전트(Web Agent) 개발의 핵심 도구입니다.

### 주요 기능

- 지능적 브라우징: 정해진 스크립트가 아니라, AI가 화면을 보고 다음 행동(클릭, 입력, 스크롤 등)을 스스로 판단합니다.
- Playwright 기반: 강력한 브라우저 자동화 도구인 Playwright를 사용하여 Chrome, Firefox 등 다양한 브라우저를 제어합니다.
- 간편한 통합: LangChain과 같은 프레임워크와 잘 연동되며, Google, OpenAI, Anthropic의 최신 모델을 비전(Vision) 기능을 통해 활용합니다.
- 멀티 탭 지원: 여러 탭을 동시에 관리하며 복잡한 정보를 비교하거나 수집할 수 있습니다.

### 작동 원리

- 관찰(Observe): 현재 웹 페이지의 HTML 구조와 스크린샷을 찍어 AI에게 전달합니다.
- 생각(Think): AI가 사용자의 목표(예: "가장 저렴한 항공권 예약해줘")와 현재 페이지 상태를 분석합니다.
- 행동(Act): 분석 결과를 바탕으로 클릭, 타이핑 등 구체적인 브라우저 액션을 실행합니다.


### CUA(Computer-Use Agent)의 일종

우리가 쓸 `Browser-use`가 바로 이 CUA 개념을 웹 브라우저 환경에 구현한 겁니다. *"다나와 가서 RTX 4090 최저가 모델 3개만 찾아와"* 라고 자연어로 던지면, 알아서 다나와 켜고, 검색하고, 클릭해서 결과를 가져옵니다.

.env 파일을 로드하여 환경변수에 등록합니다. 노트북을 위한 실행 경로를 세팅합니다.

In [1]:
import os, sys
from dotenv import load_dotenv

load_dotenv(override=True)
#load_dotenv("/mnt/d/agent_research/github-repos/home/AAWS_project/.env", override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

✅ Project Root: /workspaces/codespaces-jupyter/AAWS_project
✅ Working Directory: /workspaces/codespaces-jupyter/AAWS_project


In [2]:
import asyncio
from browser_use import Agent, Browser, ChatGoogle

import nest_asyncio

# 주피터 노트북 환경에서 비동기 루프 중복 실행 오류를 방지하기 위해 사용합니다.
nest_asyncio.apply()

# 가상 디스플레이(noVNC) 번호를 지정합니다.
# 이 설정이 있어야 GUI 브라우저가 가상 화면(:1)에 나타납니다.
os.environ["DISPLAY"] = ":1"

### 첫 번째 자율 탐색: 네이버 뉴스 요약하기

자, 이제 컴퓨터에게 첫 번째 임무를 줘보겠습니다. 
아래 코드는 아무런 HTML 태그 분석 없이, 단순히 **"네이버 뉴스에 들어가서 기사 하나를 요약해 줘"**라는 자연어 명령만을 내립니다. 
`Agent`는 다음과 같은 과정을 스스로 거칩니다:

1. 브라우저를 열고 네이버 뉴스에 접속 (Action)
2. 뉴스 목록을 확인 (Observe)
3. 흥미로운 기사를 클릭 (Act)
4. 기사 본문을 읽고 요약 (Think & Done)
Jupyter Notebook 환경에서는 비동기 충돌이 일어날 수 있으므로 `nest_asyncio`를 적용했습니다. 코드를 실행하고 브라우저가 어떻게 스스로 움직이는지, 그리고 로그(Log) 창에서 AI가 어떤 '생각(Thought)'을 거쳐 행동하는지 관찰해 보세요!

In [7]:
from browser_use import Agent, ChatGoogle
from dotenv import load_dotenv
import asyncio


async def basic_query():
    llm = ChatGoogle(model="gemini-flash-latest")
    task = "네이버 뉴스에 접속 후 보이는 첫 번째 기사를 들어가서 관련 내용을 소개해줘"

    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720} # 브라우저 크기 조절
        )

    agent = Agent(task=task, llm=llm, browser=browser)
    result = await agent.run()
    return result

if __name__ == "__main__":
    result = asyncio.run(basic_query())

INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/monitoring/telemetry.
INFO     [Agent] 🎯 Task: 네이버 뉴스에 접속 후 보이는 첫 번째 기사를 들어가서 관련 내용을 소개해줘
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-latest
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: Initial step, no previous action to evaluate. Verdict: Success
INFO     [Agent]   🧠 Memory: Starting the task to find and summarize the first article on Naver News.
INFO     [Agent]   🎯 Next goal: Navigate to Naver News to find the first article.
INFO     [Agent]   ▶️   navigate: url: https://news.naver.com, new_tab: False
INFO     [tools] 🔗 Navigated to https://news.naver.com
INFO     [Agent] 📋 Plan updated with 4 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully nav

In [8]:
# history: 여러 개의 AgentHistory 객체들이 담긴 리스트
history_list = result.history 

# 총 몇 스텝(Step)을 거쳤는지 확인
print(f"총 {len(result.history)} 스텝을 진행했습니다.")


총 4 스텝을 진행했습니다.


In [9]:
history_list[0].model_dump()

{'model_output': {'evaluation_previous_goal': 'Initial step, no previous action to evaluate. Verdict: Success',
  'memory': 'Starting the task to find and summarize the first article on Naver News.',
  'next_goal': 'Navigate to Naver News to find the first article.',
  'action': [{'navigate': {'url': 'https://news.naver.com',
     'new_tab': False}}],
  'thinking': "The user wants to see the content of the first article on Naver News. I will start by navigating to Naver News (news.naver.com). Since the user request is in Korean, I will respond in Korean as well. This is a relatively simple task, so I don't need a complex plan in todo.md yet, but I will track my progress.",
  'current_plan_item': 0,
  'plan_update': ['네이버 뉴스 접속', '첫 번째 기사 클릭', '기사 내용 추출 및 요약', '사용자에게 정보 제공']},
 'result': [{'is_done': False,
   'long_term_memory': 'Navigated to https://news.naver.com',
   'extracted_content': '🔗 Navigated to https://news.naver.com',
   'include_extracted_content_only_once': False,
   'in

In [10]:
# 마지막 스텝 정보 가져오기
last_step = result.history[-1]

print("1) LLM이 내놓은 모델의 상태 (생각/추론 과정)")
last_model_output = last_step.model_output
if last_model_output:
    print("당시 기억 : ", last_model_output.current_state.memory)  # 에이전트의 당시 기억
    print("다음 목표 : ", last_model_output.current_state.next_goal) # 에이전트의 다음 목표

print("2) LLM이 선택한 액션(명령어)")
if last_model_output and last_model_output.action:
    # 어떤 액션을 했는지 딕셔너리 형태로 변환해서 보기
    print([action.model_dump(exclude_unset=True) for action in last_model_output.action])

print("3) 에이전트가 위치했던 URL")
print(last_step.state.url) # 에이전트가 위치했던 URL

print("4) 액션 수행 후 결과")
for res in last_step.result:
    print(res.extracted_content) # 화면에서 추출한 글자 내용 
    print(res.success) # 작업 성공 여부


1) LLM이 내놓은 모델의 상태 (생각/추론 과정)
당시 기억 :  The first article on Naver News was about Park Soo-hong's brother's embezzlement case. The Supreme Court finalized a sentence of 3 years and 6 months for the brother. I have the summary ready to provide to the user.
다음 목표 :  Provide the summary of the article to the user and complete the task.
2) LLM이 선택한 액션(명령어)
[{'done': {'text': '네이버 뉴스에서 확인한 첫 번째 기사의 내용을 소개해 드립니다.\n\n**기사 제목:** [속보] ‘박수홍 출연료 횡령’ 친형에 징역 3년 6개월 확정\n\n**주요 내용:**\n방송인 박수홍 씨의 출연료와 수익금을 장기간 횡령한 혐의로 기소된 친형에 대해 대법원이 징역 3년 6개월의 실형을 최종 확정했습니다. 이로써 박수홍 씨 형제 간의 법적 공방은 친형의 유죄 판결로 마무리되었습니다.\n\n**요약:**\n- **대상:** 박수홍의 친형\n- **혐의:** 출연료 및 회사 자금 횡령\n- **결과:** 대법원, 징역 3년 6개월 확정 (2026년 2월 26일 판결)', 'success': True, 'files_to_display': []}}]
3) 에이전트가 위치했던 URL
https://n.news.naver.com/article/666/0000097432?cds=news_media_pc&type=editn
4) 액션 수행 후 결과
네이버 뉴스에서 확인한 첫 번째 기사의 내용을 소개해 드립니다.

**기사 제목:** [속보] ‘박수홍 출연료 횡령’ 친형에 징역 3년 6개월 확정

**주요 내용:**
방송인 박수홍 씨의 출연료와 수익금을 장기간 횡령한 혐의로 기소된 친형에 대해 대법원이 징역 3년 

최종 결과만 추출

browser-use 에이전트는 실행 결과를 깔끔하게 정리해주는 메서드를 제공합니다.

In [ ]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(result.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(result.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(result.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(result.urls())


### 비용 관리: 분석 비용 트래킹하기

In [11]:
from utils.cost_tracker import CostTracker

gemini-flash-lastest 모델 사용

In [ ]:
async def cost_track_browser_use1():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-flash-latest")
    task = "네이버 뉴스에 접속해서 하나의 기사를 들어가서 관련 내용을 소개해줘"
    
    # 💡 핵심 1: calculate_cost=True 옵션을 켭니다.
    agent = Agent(
        task=task, 
        llm=llm,
        calculate_cost=True
    )
    
    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use1())

gemini-flash-lastest 모델 사용

In [ ]:
async def cost_track_browser_use2():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-2.0-flash")
    task = "네이버 뉴스에 접속해서 하나의 기사를 들어가서 관련 내용을 소개해줘"
    
    # 💡 핵심 1: calculate_cost=True 옵션을 켭니다.
    agent = Agent(
        task=task, 
        llm=llm,
        calculate_cost=True
    )
    
    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use2())

In [3]:
# SJ 커스텀
import asyncio
from dotenv import load_dotenv
from browser_use import Agent, Browser, ChatGoogle
from utils.cost_tracker import CostTracker

async def cost_track_browser_use2():

    tracker = CostTracker()

    llm = ChatGoogle(model="gemini-flash-lite-latest")
    task = "네이버 뉴스에 접속 후 보이는 첫 번째 기사를 들어가서 관련 내용을 소개해줘"

    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720} # 브라우저 크기 조절
        )

    agent = Agent(task=task, llm=llm, browser=browser, calculate_cost=True)
    

    # Agent 실행 후 결과(history) 반환받기
    history = await agent.run()
    
    # ==========================================
    # 💡 핵심 2: 비용 및 토큰 사용량(UsageSummary) 출력
    # ==========================================
    if history.usage:
        print("\n📊 [토큰 및 비용 사용량 요약]")
        print(f"- 전체 사용 토큰: {history.usage.total_tokens} tokens")
        print(f"  └ 입력(Prompt) 토큰: {history.usage.total_prompt_tokens} tokens")
        print(f"  └ 출력(Completion) 토큰: {history.usage.total_completion_tokens} tokens")
        print(f"- 총 발생 비용: ${history.usage.total_cost:.6f}") # 소수점 자리수 확장

        # 트래커를 통해 파일에 누적 기록하기
        tracker.record_usage(task, history.usage)
    else:
        print("\n비용 계산에 실패했거나 측정되지 않았습니다.")
if __name__ == "__main__":
    asyncio.run(cost_track_browser_use2())



INFO     [service] Using anonymized telemetry, see https://docs.browser-use.com/development/monitoring/telemetry.


INFO     [Agent] 🎯 Task: 네이버 뉴스에 접속 후 보이는 첫 번째 기사를 들어가서 관련 내용을 소개해줘
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-lite-latest
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: N/A - Agent initialized.
INFO     [Agent]   🧠 Memory: The task is to find and summarize the first article on Naver News. Starting by searching for '네이버 뉴스' to navigate to the correct page.
INFO     [Agent]   🎯 Next goal: Search for '네이버 뉴스' using DuckDuckGo to find the Naver News homepage.
INFO     [Agent]   ▶️   search: query: 네이버 뉴스, engine: duckduckgo
INFO     [tools] 🔍  Searched Duckduckgo for '네이버 뉴스'
INFO     [Agent] 📋 Plan updated with 7 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Previous step successfully searched for '네이버 뉴스' on DuckDuckGo. Verdict: Success
INFO     [Agen


📊 [토큰 및 비용 사용량 요약]
- 전체 사용 토큰: 87927 tokens
  └ 입력(Prompt) 토큰: 76605 tokens
  └ 출력(Completion) 토큰: 11322 tokens
- 총 발생 비용: $0.009607
💰 [비용 기록 완료] 이번 작업(네이버 뉴스에 접속 후 보이는 첫 번째 기사를 들어가서 관련 내용을 소개해줘): $0.0096 / 누적 총액: $0.1069


### 심화 기능 활용하기

##### (1) 기존 브라우저 연결

지금까지는 AI가 실행될 때마다 아무 기록이 없는 '새 브라우저(샌드박스)'를 띄웠습니다. 하지만 실무에서는 이미 로그인이 다 되어 있는 내 컴퓨터의 브라우저를 그대로 조종해야 할 때가 많습니다. 

이 코드는 **현재 여러분 컴퓨터에 설치된 실제 구글 크롬**을 찾아 연결합니다. (미리 크롬 브라우저를 모두 닫아두고 실행해 주세요.)

In [19]:
Browser.list_chrome_profiles()

[]

In [22]:
def get_default_chrome_profile() -> str | None:
    profiles = Browser.list_chrome_profiles()
    
    if not profiles:
        print("❌ 설치된 크롬 프로필을 찾을 수 없습니다.")
        return None
        
    print(f"✅ 발견된 크롬 프로필: {profiles[0]['name']}")
    return profiles[0]['directory']

In [23]:
get_default_chrome_profile()

❌ 설치된 크롬 프로필을 찾을 수 없습니다.


In [27]:
async def real_browser_query():
    # 1. 내 시스템 기반의 Browser 객체 생성
    profile_path = get_default_chrome_profile()
    
    # 2. 에이전트 실행 시 생성한 브라우저 객체를 주입
    browser = Browser.from_system_chrome(profile_directory=profile_path)
    llm = ChatGoogle(model="gemini-flash-lite-latest")
    
    task = "아마존(amazon.com)에 들어가서 화이트보드 마커펜을 검색해 줘."
    
    agent = Agent(
        task=task, 
        llm=llm, 
        browser=browser
    )
    
    result = await agent.run()
    
    # 세션 종료 후 브라우저 닫기
    await browser.stop()
    return result

result = asyncio.run(real_browser_query())
print(result.final_result())

❌ 설치된 크롬 프로필을 찾을 수 없습니다.


RuntimeError: Chrome not found. Please install Chrome or use Browser() with explicit executable_path.
Expected locations:
  macOS: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
  Linux: /usr/bin/google-chrome or /usr/bin/chromium
  Windows: C:\Program Files\Google\Chrome\Application\chrome.exe

##### (2) 브라우저를 끄지 않고 연속해서 작업 (keep_alive=True)



In [31]:
profile_dir = os.path.join(os.getcwd(), "my_quotes_profile")

browser = Browser(
        headless=False,
        disable_security=True,
        window_size={'width': 1280, 'height': 720},
        user_data_dir=profile_dir,
        keep_alive=True
    )

# 2. LLM 및 에이전트 설정
llm = ChatGoogle(model="gemini-flash-lite-latest")
task = "http://quotes.toscrape.com에 접속해서 로그인해줘. 아이디와 패스워드는 김형욱, 1234야."

agent = Agent(
        task=task, 
        llm=llm,
        browser=browser)

history = await agent.run()
print(history.final_result())

INFO     [Agent] 🎯 Task: http://quotes.toscrape.com에 접속해서 로그인해줘. 아이디와 패스워드는 김형욱, 1234야.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-lite-latest


INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: Agent initialized.
INFO     [Agent]   🧠 Memory: Starting task: Navigate to http://quotes.toscrape.com and log in using username '김형욱' and password '1234'. Initializing plan.
INFO     [Agent]   🎯 Next goal: Navigate to http://quotes.toscrape.com.
INFO     [Agent]   ▶️   navigate: url: http://quotes.toscrape.com, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 8841ms) for http://quotes.toscrape.com
INFO     [tools] 🔗 Navigated to http://quotes.toscrape.com
INFO     [Agent] 📋 Plan updated with 7 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to http://quotes.toscrape.com. Verdict: Success
INFO     [Agent]   🧠 Memory: Currently on the homepage of quotes.toscrape.com. Identified the 'Login' link at index 44. Ready to proceed to the login page.
INFO     [Agent]   🎯 Next goal: Click the 'Login' link to navigate to the login page.
INFO     [A

http://quotes.toscrape.com에 사용자 이름 '김형욱'과 비밀번호 '1234'를 사용하여 성공적으로 로그인했습니다. 페이지에 'Logout' 링크가 표시되어 로그인 성공을 확인했습니다.


In [32]:
task = "http://quotes.toscrape.com에 접속해서 오른쪽 위에 'Logout' 버튼이 있는지 찾아보고, 현재 로그인된 상태가 맞는지 결과만 짧게 대답해 줘."
    
agent2 = Agent(
        task=task, 
        llm=llm,
        browser=browser)

history = await agent2.run()
print(history.final_result())

INFO     [Agent] 🎯 Task: http://quotes.toscrape.com에 접속해서 오른쪽 위에 'Logout' 버튼이 있는지 찾아보고, 현재 로그인된 상태가 맞는지 결과만 짧게 대답해 줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-lite-latest
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: N/A - Initial step.
INFO     [Agent]   🧠 Memory: Navigated to quotes.toscrape.com. Found the 'Logout' link, confirming the user is logged in.
INFO     [Agent]   🎯 Next goal: Call done action to report findings.
INFO     [Agent]   ▶️   done: text: 네, 'Logout' 버튼이 오른쪽 위에 있습니다. 현재 로그인된 상태입니다., success: True, files_to_display: None
INFO     [Agent] 
📄  Final Result: 
네, 'Logout' 버튼이 오른쪽 위에 있습니다. 현재 로그인된 상태입니다.


INFO     [Agent] ✅ Task completed successfully


네, 'Logout' 버튼이 오른쪽 위에 있습니다. 현재 로그인된 상태입니다.


In [33]:
await browser.stop() # 브라우저 세션 종료

INFO     [BrowserSession] [SessionManager] Cleared all owned data (targets, sessions, mappings)
INFO     [BrowserSession] ✅ Browser session reset complete


다음으로 로그인 정보(쿠키 등)를 파일로 저장하겠습니다.

In [ ]:
import os
from browser_use import Agent, Browser
from browser_use import ChatGoogle, ChatOpenAI

#llm = ChatGoogle(model="gemini-flash-lite-latest")
llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")


# keep_alive = True => 작업 끝나도 브라우저 종료 X
browser = Browser(
    headless=False,
    disable_security=True,
    window_size={'width': 1280, 'height': 720},
    keep_alive=True
)

agent = Agent(
    task="http://quotes.toscrape.com에 접속해서 로그인해줘. 아이디와 패스워드는 김형욱, 1234야.",
    llm=llm,
    browser=browser
)
history = await agent.run()
print("로그인 결과:", history.final_result())

# 현재 로그인 세션(쿠키, 로컬 스토리지 등)을 JSON 파일로 저장합니다.
await browser.export_storage_state('storage_state.json')

# ✅ 쿠키를 디스크에 저장하기 위해 명시적으로 종료
await browser.stop()
print("✅ 브라우저 종료 & 쿠키 저장 완료")


INFO     [Agent] 🎯 Task: http://quotes.toscrape.com에 접속해서 로그인해줘. 아이디와 패스워드는 김형욱, 1234야.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=google and model=gemini-flash-lite-latest


INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: N/A - Agent initialized.
INFO     [Agent]   🧠 Memory: Starting task to log into quotes.toscrape.com. Current URL is about:blank. Need to navigate to the target site.
INFO     [Agent]   🎯 Next goal: Navigate to http://quotes.toscrape.com.
INFO     [Agent]   ▶️   navigate: url: http://quotes.toscrape.com, new_tab: False
INFO     [tools] 🔗 Navigated to http://quotes.toscrape.com
INFO     [Agent] 📋 Plan updated with 6 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   👍 Eval: Successfully navigated to http://quotes.toscrape.com. Verdict: Success
INFO     [Agent]   🧠 Memory: Currently on the main page of quotes.toscrape.com. The next step is to click the login link to proceed with authentication using provided credentials.
INFO     [Agent]   🎯 Next goal: Click the Login link [45] to navigate to the login page.
INFO     [Agent]   ▶️   click: index: 45
INFO     [tools] 🖱️ Clicked a "Login"
INFO     [A

로그인 결과: http://quotes.toscrape.com에 성공적으로 접속하여 사용자 '김형욱'으로 로그인했습니다. 작업이 완료되었습니다.
✅ 브라우저 종료 & 쿠키 저장 완료


이제 다시 접속해보겠습니다.

In [ ]:
import os
from browser_use import Agent, Browser
from browser_use import ChatGoogle, ChatOpenAI

profile_dir = os.path.join(os.getcwd(), "my_quotes_profile")
#llm = ChatGoogle(model="gemini-flash-lite-latest")
llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")

# ✅ 같은 profile_dir로 새 브라우저 생성 → 저장된 쿠키 로드
browser2 = Browser(
    headless=False,
    disable_security=True,
    window_size={'width': 1280, 'height': 720},
    storage_state="storage_state.json"
)

agent2 = Agent(
    task="http://quotes.toscrape.com에 접속해서 오른쪽 위에 'Logout' 버튼이 있는지 찾아보고, 로그인 상태인지 짧게 대답해줘.",
    llm=llm,
    browser=browser2
)
history2 = await agent2.run()
print("로그인 상태:", history2.final_result())


INFO     [Agent] 🎯 Task: http://quotes.toscrape.com에 접속해서 오른쪽 위에 'Logout' 버튼이 있는지 찾아보고, 로그인 상태인지 짧게 대답해줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=openai and model=gpt-5-mini-2025-08-07


INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: No previous browsing actions performed yet; starting fresh (neutral).
INFO     [Agent]   🧠 Memory: Starting with no pages visited. Goal: visit http://quotes.toscrape.com and check for 'Logout' at top-right to tell if logged in.
INFO     [Agent]   🎯 Next goal: Open http://quotes.toscrape.com in a new tab and load the page so I can inspect the top-right for a 'Logout' button.
INFO     [Agent]   ▶️   navigate: url: http://quotes.toscrape.com, new_tab: True
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 8625ms) for http://quotes.toscrape.com
INFO     [tools] 🔗  Opened new tab with url http://quotes.toscrape.com
INFO     [Agent] 📋 Plan updated with 3 steps
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   ❔ Eval: 성공: 사이트를 열고 페이지 상단 오른쪽에 'Logout' 요소가 있는지 확인하여 찾았습니다.
INFO     [Agent]   🧠 Memory: quotes.toscrape.com 메인 페이지를 열었고, 상단 오른쪽에 'Logout' 링크([44])가 보이므로 현재 로그인 상태로 확인됨. 페이지에 팝업/오버레이는 없었음.


로그인 상태: 페이지 상단 오른쪽에 'Logout' 버튼이 보입니다. 현재 로그인 상태입니다.


##### (3) 데이터 form 전송

이번에는 정보를 폼에 채워서 전송해봅시다.

In [41]:
async def run_form_filling():
    # 1. 빈 브라우저 객체 생성

    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절)
    )
    
    # 2. LLM 및 에이전트 설정
    #llm = ChatGoogle(model="gemini-flash-lite-latest")
    llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")


    task = """
    https://httpbin.org/forms/post 에 접속해서 다음 정보로 폼을 채워줘:
    - 이름: 홍길동
    - 전화번호: 010-1234-5678
    - 이메일: test@example.com
    - 사이즈: Medium / 토핑: cheese / 시간: 지금
    - 의견: Browser-use 테스트 중입니다.
    
    폼을 제출한 후 서버로부터 받은 응답 내용을 요약해서 알려줘.
    """
    
    agent = Agent(
        task=task, 
        llm=llm,
        browser=browser)
    
    # 3. Agent 실행 후 결과(history) 반환받기
    history = await agent.run()

    # 4. 브라우저 닫기
    await browser.stop()
    
    return history
if __name__ == '__main__':
    history = asyncio.run(run_form_filling())

INFO     [Agent] 🔗 Found URL in task: https://httpbin.org/forms/post, adding as initial action...
INFO     [Agent] 🎯 Task: 
    https://httpbin.org/forms/post 에 접속해서 다음 정보로 폼을 채워줘:
    - 이름: 홍길동
    - 전화번호: 010-1234-5678
    - 이메일: test@example.com
    - 사이즈: Medium / 토핑: cheese / 시간: 지금
    - 의견: Browser-use 테스트 중입니다.

    폼을 제출한 후 서버로부터 받은 응답 내용을 요약해서 알려줘.
    
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=openai and model=gpt-5-mini-2025-08-07


INFO     [Agent]   ▶️   navigate: url: https://httpbin.org/forms/post, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 8848ms) for https://httpbin.org/forms/post
INFO     [tools] 🔗 Navigated to https://httpbin.org/forms/post
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: Success: the httpbin form page is loaded and form elements with indexes were identified.
INFO     [Agent]   🧠 Memory: Located form fields with indexes: custname [7], custtel [8], custemail [9], size medium [11], topping cheese [14], delivery time [18], comments [17], submit [97]. No popups blocking the page.
INFO     [Agent]   🎯 Next goal: Fill the form fields with the user's data and submit the form by clicking the Submit order button.
INFO     [Agent]   ▶️  [1/5] input: index: 7, text: 홍길동, clear: True
INFO     [BrowserSession] ⌨️ Typed "홍길동" into element with index 7
INFO     [Agent]   ▶️  [2/5] input: index: 8, text: 010-1234-5678, clear: True
INFO     [BrowserSes

In [42]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(history.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(history.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(history.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(history.urls())


요청하신 작업을 완료했습니다.

요약 (한국어):
- 제출된 폼 필드:
  - 이름(custname): 홍길동
  - 전화번호(custtel): 010-1234-5678
  - 이메일(custemail): test@example.com
  - 사이즈(size): medium (사용자 요청: Medium)
  - 토핑(topping): cheese (사용자 요청: cheese)
  - 배달시간(delivery): 12:00 (사용자 요청: 지금 — 폼에서는 '12:00'로 입력됨)
  - 의견(comments): Browser-use 테스트 중입니다.

- 서버 응답 관련 정보:
  - 요청 URL: https://httpbin.org/post
  - 요청을 보낸 클라이언트 IP(origin): 23.97.62.144
  - 요청 Content-Type: application/x-www-form-urlencoded
  - 응답에 포함된 form 객체에 위의 모든 필드가 정확히 반영되어 있습니다.

참고: 서버가 반환한 전체 JSON(요청 헤더와 form 내용 포함)은 현재 탭의 페이지에 표시되어 있으며, 원하시면 전체 원문을 그대로 제공하거나 파일로 저장해 드릴 수 있습니다.

성공여부: true
True
[None, None, None, None]
['https://httpbin.org/forms/post', 'https://httpbin.org/forms/post', 'https://httpbin.org/forms/post', 'https://httpbin.org/post']


##### (4) 페이지 내 데이터 추출

기본적인 데이터 추출을 명령해봅니다.

In [43]:
async def run_data_extraction():
    
    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절)
    )
    
    
    #llm = ChatGoogle(model="gemini-flash-lite-latest")
    llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")
    
    # 5개의 명언, 저자, 태그를 특정 형식으로 추출 요청
    task = """
    https://quotes.toscrape.com/ 에서 다음 정보를 추출해줘:
    - 페이지에 있는 첫 5개의 명언
    - 각 명언의 저자와 태그들
    
    결과는 '명언 1: [내용] - 저자: [이름] - 태그: [태그들]' 형식으로 정리해줘.
    """
    
    agent = Agent(task=task, llm=llm, browser=browser)
    history = await agent.run()
    return history

if __name__ == '__main__':
    history = asyncio.run(run_data_extraction())

INFO     [Agent] 🔗 Found URL in task: https://quotes.toscrape.com/, adding as initial action...
INFO     [Agent] 🎯 Task: 
    https://quotes.toscrape.com/ 에서 다음 정보를 추출해줘:
    - 페이지에 있는 첫 5개의 명언
    - 각 명언의 저자와 태그들

    결과는 '명언 1: [내용] - 저자: [이름] - 태그: [태그들]' 형식으로 정리해줘.
    
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=openai and model=gpt-5-mini-2025-08-07
INFO     [Agent]   ▶️   navigate: url: https://quotes.toscrape.com/, new_tab: False
INFO     [tools] 🔗 Navigated to https://quotes.toscrape.com/
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   👍 Eval: Success — the page loaded and the quotes and tags are visible in the browser_state and screenshot.
INFO     [Agent]   🧠 Memory: Page https://quotes.toscrape.com/ loaded and first several quote entries are visible. Identified and extracted the first five quotes with authors 

In [44]:
history

AgentHistoryList(all_results=[ActionResult(is_done=False, success=None, judgement=None, error=None, attachments=None, images=None, long_term_memory='Found initial url and automatically loaded it. Navigated to https://quotes.toscrape.com/', extracted_content='🔗 Navigated to https://quotes.toscrape.com/', include_extracted_content_only_once=False, metadata=None, include_in_memory=False), ActionResult(is_done=True, success=True, judgement=JudgementResult(reasoning="User asked to extract from https://quotes.toscrape.com/ the first 5 quotes on the page, and for each quote provide the author and tags in the specified format. The agent navigated to the requested URL and returned five items, each containing the quote text, the author, and the tag list. The extracted content matches the visible page content (quotes and tags seen in the screenshot) and is formatted exactly as the user requested: '명언 N: [내용] - 저자: [이름] - 태그: [태그들]'. Tool usage was minimal but appropriate: a single navigation acti

In [45]:
# 최종 추출된 결과 텍스트만 쏙 빼오기
print(history.final_result())

# 작업이 성공적으로 잘 끝났는지 (True / False / None)
print(history.is_successful())

# 작업 중 발생한 에러 메시지들만 리스트로 모아오기
print(history.errors())

# 방문했던 모든 URL들 순서대로 확인하기
print(history.urls())


명언 1: “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” - 저자: Albert Einstein - 태그: change, deep-thoughts, thinking, world

명언 2: “It is our choices, Harry, that show what we truly are, far more than our abilities.” - 저자: J.K. Rowling - 태그: abilities, choices

명언 3: “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” - 저자: Albert Einstein - 태그: inspirational, life, live, miracle, miracles

명언 4: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” - 저자: Jane Austen - 태그: aliteracy, books, classic, humor

명언 5: “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” - 저자: Marilyn Monroe - 태그: be-yourself, inspirational
True
[None, None]
['https://quotes.toscrape.com/', 'https://quotes.toscrape.com/']


##### (5) 에이전트의 빠른 브라우징 작업

에이전트의 불필요한 생각 시간을 줄이고 브라우저 동작 속도를 극대화하는 설정입니다.

- flash_mode=True: AI의 내부 사고 과정을 생략하여 응답 속도를 높입니다.
- Browser: 페이지 로딩 대기 시간과 액션 간격을 0.1초로 단축합니다.
- extend_system_message: AI에게 "최대한 짧고 빠르게 행동하라"는 지침을 추가합니다.

In [46]:
from browser_use import BrowserProfile

async def run_fast_agent():
   
    #llm = ChatGoogle(model="gemini-flash-lite-latest")
    llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")
    
    # 1. 속도 최적화 브라우저 설정   
    browser = Browser(
            headless=False,  # VNC 화면에 브라우저 창 표시
            disable_security=True, # 보안 정책(CORS 등)으로 인한 탐색 제한을 방지
            window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절
            wait_between_actions=0.1,        # 동작 간격 단축
            minimum_wait_page_load_time=0.1 # 최소 대기 시간 단축
    )


    # 2. 모델에게 내리는 '빨리빨리' 명령
    optimization_prompt = "최대한 간결하게 응답하고, 가능한 여러 동작을 한 번에 실행해서 목표에 빠르게 도달해."

    agent = Agent(
        task="네이버 뉴스 상위 3개 기사 제목만 빠르게 뽑아줘.",
        llm=llm,
        flash_mode=True,                # 사고 과정 생략
        browser=browser,
        extend_system_message=optimization_prompt
    )
    
    await agent.run()

if __name__ == '__main__':
    asyncio.run(run_fast_agent())

INFO     [Agent] 🎯 Task: 네이버 뉴스 상위 3개 기사 제목만 빠르게 뽑아줘.
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=openai and model=gpt-5-mini-2025-08-07
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   🧠 Memory: No previous page loaded. User asked (in Korean) for the top 3 Naver News article headlines. Next step: open Naver News main page to extract the top headlines. Immediate action: navigate to https://news.naver.com and then find the top 3 article titles.
INFO     [Agent]   ▶️   navigate: url: https://news.naver.com, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 10338ms) for https://news.naver.com
INFO     [tools] 🔗 Navigated to https://news.naver.com
INFO     [Agent] 

INFO     [Agent] 📍 Step 2:
INFO     [Agent]   🧠 Memory: Step successful: page loaded and I located top headlines in the main content. Remem

##### (6) 비디오 녹화

작업 과정을 녹화합니다. AI가 어떻게 움직였는지 사후에 검토하거나 디버깅할 때 매우 유용합니다.

In [47]:
import asyncio
from pathlib import Path
from browser_use import Agent, Browser, ChatGoogle, ChatOpenAI
from browser_use.browser import BrowserProfile
from dotenv import load_dotenv

load_dotenv()

async def main():
    # 1. 영상이 저장될 폴더 설정
    video_dir = Path('./recordings')
    video_dir.mkdir(exist_ok=True)
    
    # 2. 브라우저 설정 (핵심: record_video_dir)
    # 구조를 따름
    browser = Browser(
        headless=True, # VNC 화면에 브라우저 창 표시하지 않습니다.
        window_size={'width': 1280, 'height': 720}, # 브라우저 크기 조절)
        wait_between_actions=0.1,        # 동작 간격 단축
        minimum_wait_page_load_time=0.1, # 최소 대기 시간 단축
        record_video_dir=video_dir,  # 영상 저장 경로 지정
    )

    # 4. LLM 및 에이전트 설정
    #llm = ChatGoogle(model="gemini-flash-lite-latest")
    llm = ChatOpenAI(model="gpt-5-mini-2025-08-07")
    
    task = "https://www.naver.com 에 오늘의 날씨에 대해 찾아보고 알려줘"

    agent = Agent(
        task=task,
        llm=llm,
        browser=browser,
    )

    print(f"🚀 작업을 시작합니다. 녹화본은 '{video_dir}' 폴더에 저장됩니다.")

    try:
        # 에이전트 실행 (단계 제한을 둬서 테스트)
        return await agent.run(max_steps=10)
    finally:
        print(f"✅ 작업 완료! '{video_dir}' 폴더에서 mp4 파일을 확인하세요.")

if __name__ == '__main__':
    history = asyncio.run(main())

INFO     [Agent] 🔗 Found URL in task: https://www.naver.com, adding as initial action...
INFO     [Agent] 🎯 Task: https://www.naver.com 에 오늘의 날씨에 대해 찾아보고 알려줘
INFO     [Agent] 📦 Newer version available: 0.12.0 (current: 0.11.13). Upgrade with: uv add browser-use==0.12.0
INFO     [Agent] Starting a browser-use agent with version 0.11.13, with provider=openai and model=gpt-5-mini-2025-08-07


🚀 작업을 시작합니다. 녹화본은 'recordings' 폴더에 저장됩니다.


INFO     [BrowserSession] Setting viewport to 1280x720 with device scale factor 1.0 whereas original device scale factor was 1.0
INFO     [BrowserSession] 📹 Started/Switched video recording to target 1E0FC888498B71BEE6630D821CDDE324
INFO     [Agent]   ▶️   navigate: url: https://www.naver.com, new_tab: False
WARNING  [BrowserSession] ⚠️ Page readiness timeout (8.0s, 12854ms) for https://www.naver.com
INFO     [tools] 🔗 Navigated to https://www.naver.com
INFO     [Agent] 

INFO     [Agent] 📍 Step 1:
INFO     [Agent]   ❔ Eval: Succeeded: navigated to Naver home and located the weather widget in the page state and screenshot.
INFO     [Agent]   🧠 Memory: On Naver main page, a weather widget is visible (element [234] '날씨' and [236] showing 서울 을지로1가 13.1℃ 구름많음, 최고 15℃). Next step is to open the full weather page for today's details.
INFO     [Agent]   🎯 Next goal: Click the '날씨' link (element [234]) to open the full weather details page on Naver.
INFO     [Agent]   ▶️   click: index: 234
IN

✅ 작업 완료! 'recordings' 폴더에서 mp4 파일을 확인하세요.


In [48]:
history.final_result()

"요약(한국어):\n\n- 위치: 중구 을지로1가 (네이버 날씨 기준)\n- 현재기온: 13.1°C, 체감: 13.1°C\n- 하늘상태: 구름많음\n- 오늘 최저/최고: 최저 1°C / 최고 15°C\n- 바람: 남동풍 약 2.3 m/s\n- 습도: 38%\n- 대기질: 미세먼지 보통 / 초미세먼지 보통\n- 일출/일몰: 일출 07:09 / 일몰 18:22\n- 시간별 요약: 낮 시간대에는 맑은/구름낀 구간이 섞이며 오후 일부 시간대 강수확률 20~30% 수준, 저녁부터 밤에 구름이 많아짐(세부 시간별 기온은 페이지의 시간표 참고).\n- 업데이트 시각: 페이지에 표기된 최근 업데이트 약 13:29\n\n한줄요약: 현재 서울(을지로1가)은 구름이 많은 상태에 기온은 약 13°C로 비교적 포근하며, 오늘 낮 최고는 15°C, 미세먼지는 '보통' 수준입니다. 외출 시 가벼운 겉옷 권장합니다."